In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, ConfusionMatrixDisplay
from transformers import BertTokenizer, BertModel
from datasets import load_dataset
import torch
from scipy.spatial.distance import cosine
import json

# Configuramos un estilo para los gráficos
plt.style.use('seaborn-v0_8-whitegrid')

c:\Users\losfe\Escritorio\Martin\Facultad\Pre cambio pc\UBA\Aprendizaje Automatico\AA2\notebooks\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [30]:
dsa = load_dataset("ylacombe/google-argentinian-spanish", "male")
dsaf = load_dataset("ylacombe/google-argentinian-spanish", "female")

c:\Users\losfe\Escritorio\Martin\Facultad\Pre cambio pc\UBA\Aprendizaje Automatico\AA2\notebooks\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\losfe\.cache\huggingface\hub\datasets--ylacombe--google-argentinian-spanish. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Generating train split: 100%

In [ ]:
pd.set_option('display.max_colwidth', None)

dfa = dsa["train"].to_pandas()
dfaf = dsaf["train"].to_pandas()
dfa = dfa.concat(dfaf)
dfa2 = dfa.drop("audio", axis=1)
dfa2 = dfa2.drop("speaker_id", axis=1)
dfa2.describe()

,text
count,2636
unique,1794
top,No te entiendo nada
freq,35


In [ ]:
from datasets import load_dataset

dsc = load_dataset("ylacombe/google-chilean-spanish", "male")

c:\Users\losfe\Escritorio\Martin\Facultad\Pre cambio pc\UBA\Aprendizaje Automatico\AA2\notebooks\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\losfe\.cache\huggingface\hub\datasets--ylacombe--google-chilean-spanish. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Generating train split: 100%|███

In [ ]:
dfc = dsc["train"].to_pandas()
dfc2 = dfc.drop("audio", axis=1)
dfc2 = dfc2.drop("speaker_id", axis=1)
dfc2.describe()

,text
count,2636
unique,1794
top,No te entiendo nada
freq,35


In [31]:
preguntas = []
contextos = []

with open('./SQAC/dev.json', 'r') as file:
    data = json.load(file)  # json.load parses the file into a Python dictionary
    data2 = data['data']

    for elemento in data2:
        parrafos = elemento['paragraphs']

        for parrafo in parrafos:

            contexto = parrafo['context']
            contextos.append(contexto)

            questions = parrafo['qas']
            for pregunta in questions:
                preg = pregunta['question']
                preguntas.append(preg)

print(len(contextos))
print(len(preguntas))

preguntas

615
1864


['¿Cuántas lenguas aborígenes existen en México?',
 '¿Cuál es el idioma más hablado en México?',
 '¿Qué lenguajes son considerados lenguas nacionales en México?',
 '¿Con qué intención se han estudiado los dialectos del castellano que se hablan en América?',
 '¿Qué zonas abarca la primera rama del español mexicano-centroamericano?',
 '¿Cuántas subdivisiones tiene el español mexicano-centroamericano según Moreno Fernández?',
 '¿Cuál es la proporción de hablantes de lenguas amerindias en Guatemala?',
 '¿Qué posición ocupa el náhuatl por tamaño de hablantes respecto a las lenguas indígenas de América?',
 '¿Cuál es el único idioma indígena mexicano con más de un millón de hablantes?',
 '¿Qué profesionales de la comunicación han muerto en México?',
 '¿Quién fue el árbitro del partido?',
 '¿Cuándo marcó gol Robinho?',
 '¿Dónde jugaron Holanda y Brasil?',
 '¿Dónde tienen lugar los hechos narrados?',
 '¿Qué se ha revocado en este territorio?',
 '¿De qué empresas quiere tomar el control el grupo

In [ ]:
data_espanol_martin2 = load_dataset("openbmb/DCAD-2000", name="spa_Latn", split="train")

c:\Users\losfe\Escritorio\Martin\Facultad\Pre cambio pc\UBA\Aprendizaje Automatico\AA2\notebooks\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\losfe\.cache\huggingface\hub\datasets--openbmb--DCAD-2000. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)